# GLM-4-Voice 音频对话测试

本 Notebook 用于测试智谱 GLM-4-Voice 模型的音频对话功能。

> **注意**：GLM-4-Voice 是专门的语音模型，不支持思考模式，不返回思维链。

功能包括：
- 文本对话
- 语音输入
- 语音输出
- 多轮语音对话
- 语音识别
- 语音合成
- 错误处理

## 1. 环境配置

In [ ]:
# 安装必要的依赖（如果尚未安装）
# !pip install requests -q

In [ ]:
import requests
import base64
import json
import os
from IPython.display import Audio, display
import time

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
API_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"

# 请求头
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

print("✅ 环境配置完成")
print(f"API URL: {API_URL}")

### 辅助函数

In [ ]:
# 辅助函数：将音频文件编码为 base64
def encode_audio(file_path):
    """
    将音频文件转换为 base64 编码字符串
    
    Args:
        file_path: 音频文件路径
    
    Returns:
        base64 编码的字符串
    """
    try:
        with open(file_path, "rb") as audio_file:
            encoded = base64.b64encode(audio_file.read()).decode('utf-8')
        print(f"✅ 成功编码音频文件: {file_path}")
        print(f"   编码长度: {len(encoded)} 字符")
        return encoded
    except FileNotFoundError:
        print(f"❌ 文件未找到: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 编码失败: {str(e)}")
        return None

# 辅助函数：保存 base64 音频数据到文件
def save_audio(base64_data, output_path, display_audio=True):
    """
    将 base64 编码的音频数据保存为文件
    
    Args:
        base64_data: base64 编码的音频数据
        output_path: 输出文件路径
        display_audio: 是否在 Notebook 中播放音频
    """
    try:
        audio_data = base64.b64decode(base64_data)
        with open(output_path, "wb") as f:
            f.write(audio_data)
        print(f"✅ 音频已保存到: {output_path}")
        print(f"   文件大小: {len(audio_data)} 字节")
        
        if display_audio:
            print("🔊 音频预览:")
            display(Audio(output_path))
        return True
    except Exception as e:
        print(f"❌ 保存失败: {str(e)}")
        return False

# 辅助函数：解析并显示响应内容
def parse_response(response_data, save_audio_path=None):
    """
    解析 API 响应并显示内容
    
    Args:
        response_data: API 返回的 JSON 数据
        save_audio_path: 如果有音频，保存的路径
    """
    if "error" in response_data:
        print(f"❌ API 错误: {response_data['error']}")
        return None
    
    if "choices" not in response_data:
        print(f"⚠️  响应格式异常: {response_data}")
        return None
    
    content = response_data["choices"][0]["message"]["content"]
    result = {"text": [], "audio": []}
    
    # 处理字符串类型或列表类型的 content
    if isinstance(content, str):
        print(f"📝 文本回复:")
        print(content)
        result["text"].append(content)
    elif isinstance(content, list):
        for item in content:
            if item["type"] == "text":
                print(f"📝 文本回复:")
                print(item["text"])
                result["text"].append(item["text"])
            elif item["type"] == "audio":
                print(f"🔊 收到音频数据")
                audio_data = item["audio"]["data"]
                result["audio"].append(audio_data)
                if save_audio_path:
                    save_audio(audio_data, save_audio_path)
    
    # 显示 Token 使用情况
    if "usage" in response_data:
        usage = response_data["usage"]
        print(f"\n📊 Token 使用情况:")
        print(f"   输入: {usage.get('prompt_tokens', 'N/A')}")
        print(f"   输出: {usage.get('completion_tokens', 'N/A')}")
        print(f"   总计: {usage.get('total_tokens', 'N/A')}")
    
    return result

print("✅ 辅助函数定义完成")

## 2. GLM-4-Voice 文本对话测试

GLM-4-Voice 不仅支持语音，也支持纯文本输入。

**注意**：GLM-4-Voice 不支持思考模式（thinking 参数），不返回思维链（reasoning_content）。

In [ ]:
def text_chat(message, model="glm-4-voice"):
    """
    发送文本消息进行对话
    
    Args:
        message: 用户输入的文本消息
        model: 使用的模型名称
    """
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": message}
        ]
    }
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 测试 1: 简单的问候
print("=== 测试 1: 简单问候 ===" + "\n")
result = text_chat("你好，请简单介绍一下你自己")
parse_response(result)

In [ ]:
# 测试 2: 多轮文本对话
print("=== 测试 2: 多轮文本对话 ===" + "\n")

messages = [
    {"role": "user", "content": "你喜欢什么运动？"}
]

payload = {
    "model": "glm-4-voice",
    "messages": messages
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()
parse_response(result)

# 添加回复到历史
messages.append({
    "role": "assistant", 
    "content": result["choices"][0]["message"]["content"]
})

# 继续对话
print("\n--- 用户追问 ---\n")
messages.append({"role": "user", "content": "为什么喜欢这个运动？"})

payload["messages"] = messages
response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()
parse_response(result)

## 3. 语音输入测试

测试将音频文件作为输入发送给模型。

> **注意**: 本测试需要准备音频文件。如果没有现成的音频文件，可以先创建一个简单的测试音频。

In [ ]:
# 创建示例音频文件（使用 Python 生成一个简单正弦波音频用于测试）
# 如果没有音频文件，可以运行此代码创建测试音频

try:
    import numpy as np
    from scipy.io import wavfile
    
    # 生成 2 秒的 440Hz 正弦波（A4 音）
    sample_rate = 16000
    duration = 2
    t = np.linspace(0, duration, int(sample_rate * duration))
    frequency = 440
    audio_data = np.sin(2 * np.pi * frequency * t) * 0.3
    
    # 转换为 16-bit PCM
    audio_int16 = (audio_data * 32767).astype(np.int16)
    
    # 保存
    test_audio_path = "test_input.wav"
    wavfile.write(test_audio_path, sample_rate, audio_int16)
    print(f"✅ 已创建测试音频: {test_audio_path}")
    print(f"   采样率: {sample_rate} Hz")
    print(f"   时长: {duration} 秒")
    
    # 播放测试音频
    display(Audio(test_audio_path))
    
except ImportError:
    print("⚠️ 需要安装 numpy 和 scipy 来生成测试音频")
    print("   运行: pip install numpy scipy")
    print("\n请准备自己的音频文件，命名为 test_input.wav")

In [ ]:
# 检查音频文件是否存在
test_audio_path = "test_input.wav"

if not os.path.exists(test_audio_path):
    print(f"⚠️  音频文件 {test_audio_path} 不存在")
    print("请准备音频文件或运行上方代码生成测试音频")
else:
    print(f"✅ 找到音频文件: {test_audio_path}")
    print(f"   文件大小: {os.path.getsize(test_audio_path)} 字节")
    
    # 编码音频
    audio_base64 = encode_audio(test_audio_path)
    
    # 显示音频
    print("\n🔊 输入音频:")
    display(Audio(test_audio_path))

In [ ]:
# 发送语音输入测试
def audio_input_chat(audio_base64, audio_format="wav", text_prompt=None):
    """
    发送语音输入进行对话
    
    Args:
        audio_base64: base64 编码的音频数据
        audio_format: 音频格式 (wav 或 mp3)
        text_prompt: 可选的文本提示
    """
    content = [
        {
            "type": "input_audio",
            "input_audio": {
                "data": audio_base64,
                "format": audio_format
            }
        }
    ]
    
    if text_prompt:
        content.insert(0, {"type": "text", "text": text_prompt})
    
    payload = {
        "model": "glm-4-voice",
        "messages": [
            {"role": "user", "content": content}
        ]
    }
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 测试语音输入
print("=== 语音输入测试 ===" + "\n")

if 'audio_base64' in dir() and audio_base64:
    result = audio_input_chat(audio_base64)
    parse_response(result)
else:
    print("❌ 没有可用的音频数据，请先准备音频文件")

## 4. 语音输出测试

测试让模型返回语音回复，并保存播放。

In [ ]:
# 发送文本请求，获取语音回复
print("=== 语音输出测试 ===" + "\n")

# 请求语音回复
payload = {
    "model": "glm-4-voice",
    "messages": [
        {
            "role": "user", 
            "content": "请用语音朗读：你好，我是 GLM-4-Voice，我可以理解语音并进行语音回复。"
        }
    ]
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()

# 解析响应并保存音频
output_path = "output_response.wav"
parse_result = parse_response(result, save_audio_path=output_path)

In [ ]:
# 再次测试不同的文本
print("=== 语音合成测试 - 古诗朗诵 ===" + "\n")

payload = {
    "model": "glm-4-voice",
    "messages": [
        {
            "role": "user", 
            "content": "请朗读李白的《静夜思》"
        }
    ]
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()
parse_response(result, save_audio_path="poem_recitation.wav")

## 5. 多轮语音对话测试

测试保持上下文的多轮语音对话。

In [ ]:
# 多轮语音对话
print("=== 多轮语音对话测试 ===" + "\n")

# 初始化对话历史
conversation_history = []
round_num = 0

def add_message(role, content):
    """添加消息到对话历史"""
    conversation_history.append({"role": role, "content": content})

def send_conversation():
    """发送当前对话历史并获取回复"""
    payload = {
        "model": "glm-4-voice",
        "messages": conversation_history
    }
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 第一轮：用户发送文本
round_num += 1
print(f"\n{'='*20} 第 {round_num} 轮 {'='*20}\n")

user_input = "我想学习编程，有什么建议吗？"
print(f"👤 用户: {user_input}\n")

add_message("user", user_input)
result = send_conversation()
parse_response(result, save_audio_path=f"round_{round_num}_response.wav")

# 添加助手回复到历史
if "choices" in result:
    assistant_content = result["choices"][0]["message"]["content"]
    add_message("assistant", assistant_content)

In [ ]:
# 第二轮：用户追问（假设有语音输入）
round_num += 1
print(f"\n{'='*20} 第 {round_num} 轮 {'='*20}\n")

# 这里使用文本模拟语音输入
user_input = "Python 和 JavaScript 哪个更适合初学者？"
print(f"👤 用户: {user_input}\n")

add_message("user", user_input)
result = send_conversation()
parse_response(result, save_audio_path=f"round_{round_num}_response.wav")

if "choices" in result:
    assistant_content = result["choices"][0]["message"]["content"]
    add_message("assistant", assistant_content)

In [ ]:
# 第三轮：继续对话
round_num += 1
print(f"\n{'='*20} 第 {round_num} 轮 {'='*20}\n")

user_input = "好的，我决定学 Python，如何开始？"
print(f"👤 用户: {user_input}\n")

add_message("user", user_input)
result = send_conversation()
parse_response(result, save_audio_path=f"round_{round_num}_response.wav")

if "choices" in result:
    assistant_content = result["choices"][0]["message"]["content"]
    add_message("assistant", assistant_content)

print(f"\n{'='*50}\n")
print(f"✅ 多轮对话完成，共 {round_num} 轮")
print(f"📊 对话历史消息数: {len(conversation_history)}")

## 6. 语音识别测试（语音转文字）

测试将语音转换为文字的功能。

In [ ]:
# 语音识别测试
print("=== 语音识别测试 ===" + "\n")

def speech_to_text(audio_base64, audio_format="wav"):
    """
    语音识别：将语音转换为文字
    
    通过发送语音并要求模型转录文字来实现
    """
    content = [
        {
            "type": "text",
            "text": "请转录这段语音的内容，只输出文字，不要其他解释。"
        },
        {
            "type": "input_audio",
            "input_audio": {
                "data": audio_base64,
                "format": audio_format
            }
        }
    ]
    
    payload = {
        "model": "glm-4-voice",
        "messages": [
            {"role": "user", "content": content}
        ]
    }
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 如果有测试音频，进行语音识别
if 'audio_base64' in dir() and audio_base64:
    result = speech_to_text(audio_base64)
    print("🔊 输入音频内容识别结果:\n")
    parse_response(result)
else:
    print("⚠️  没有可用音频进行语音识别测试")

In [ ]:
# 测试不同语言的语音识别（如果有对应音频）
print("=== 多语言语音识别 ===" + "\n")

# 注意：这里假设使用之前生成的测试音频
# 实际使用时需要准备对应语言的音频文件

test_scenarios = [
    ("test_input.wav", "测试音频"),
]

for audio_file, description in test_scenarios:
    print(f"\n--- {description} ---")
    if os.path.exists(audio_file):
        audio_b64 = encode_audio(audio_file)
        if audio_b64:
            result = speech_to_text(audio_b64)
            parse_response(result)
    else:
        print(f"⚠️  文件 {audio_file} 不存在，跳过")

## 7. 语音合成测试（文字转语音）

测试将文字转换为语音的功能。

In [ ]:
# 语音合成测试
print("=== 语音合成测试 ===" + "\n")

def text_to_speech(text):
    """
    语音合成：将文字转换为语音
    
    Args:
        text: 要转换为语音的文本
    """
    payload = {
        "model": "glm-4-voice",
        "messages": [
            {
                "role": "user",
                "content": f"请将以下内容转换为语音：{text}"
            }
        ]
    }
    
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 测试不同的文本内容
test_texts = [
    "欢迎使用 GLM-4-Voice 语音合成服务。",
    "今天天气真好，适合出去走走。",
    "Hello, this is a test of English speech synthesis.",
]

for i, text in enumerate(test_texts, 1):
    print(f"\n{'='*20} 测试 {i} {'='*20}\n")
    print(f"📝 原文: {text}\n")
    
    result = text_to_speech(text)
    parse_response(result, save_audio_path=f"tts_test_{i}.wav")
    
    time.sleep(0.5)  # 避免请求过快

In [ ]:
# 长文本语音合成测试
print("=== 长文本语音合成测试 ===" + "\n")

long_text = """
人工智能正在深刻改变我们的生活方式。
从智能助手到自动驾驶，从医疗诊断到教育辅助，
AI 技术无处不在。GLM-4-Voice 作为先进的语音模型，
能够理解语音、生成语音，实现自然的人机交互。
""".strip()

print(f"📝 长文本 ({len(long_text)} 字符):")
print(long_text[:100] + "...\n")

result = text_to_speech(long_text)
parse_response(result, save_audio_path="tts_long_text.wav")

## 8. 错误处理测试

测试各种错误情况和异常处理。

In [ ]:
# 错误处理测试
print("=== 错误处理测试 ===" + "\n")

# 测试 1: 无效的 API Key
print("\n--- 测试 1: 无效的 API Key ---\n")

invalid_headers = {
    "Authorization": "Bearer invalid_api_key",
    "Content-Type": "application/json"
}

payload = {
    "model": "glm-4-voice",
    "messages": [{"role": "user", "content": "你好"}]
}

response = requests.post(API_URL, headers=invalid_headers, json=payload)
print(f"状态码: {response.status_code}")
print(f"响应: {response.text[:500]}")

In [ ]:
# 测试 2: 无效的模型名称
print("\n--- 测试 2: 无效的模型名称 ---\n")

payload = {
    "model": "invalid-model-name",
    "messages": [{"role": "user", "content": "你好"}]
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()

if "error" in result:
    print(f"❌ 预期的错误:")
    print(json.dumps(result["error"], indent=2, ensure_ascii=False))
else:
    print(f"⚠️  意外响应: {result}")

In [ ]:
# 测试 3: 空消息
print("\n--- 测试 3: 空消息 ---\n")

payload = {
    "model": "glm-4-voice",
    "messages": []
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()

if "error" in result:
    print(f"❌ 预期的错误:")
    print(json.dumps(result["error"], indent=2, ensure_ascii=False))

In [ ]:
# 测试 4: 无效的 base64 音频数据
print("\n--- 测试 4: 无效的 base64 音频数据 ---\n")

payload = {
    "model": "glm-4-voice",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": "invalid_base64_data!!!",
                        "format": "wav"
                    }
                }
            ]
        }
    ]
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()

if "error" in result:
    print(f"❌ 预期的错误:")
    print(json.dumps(result["error"], indent=2, ensure_ascii=False))
else:
    print(f"⚠️  响应 (可能没有错误):")
    print(json.dumps(result, indent=2, ensure_ascii=False)[:500])

In [ ]:
# 测试 5: 网络超时处理
print("\n--- 测试 5: 网络超时处理 ---\n")

try:
    # 设置较短的超时时间
    payload = {
        "model": "glm-4-voice",
        "messages": [{"role": "user", "content": "你好"}]
    }
    
    response = requests.post(API_URL, headers=headers, json=payload, timeout=0.001)
    print("✅ 请求成功")
except requests.exceptions.Timeout:
    print("⏱️  请求超时（预期的超时错误）")
except Exception as e:
    print(f"❌ 其他错误: {type(e).__name__}: {str(e)}")

In [ ]:
# 测试 6: 超长的输入
print("\n--- 测试 6: 超长输入测试 ---\n")

# 生成超长文本
long_input = "测试 " * 10000  # 约 50000 字符
print(f"输入长度: {len(long_input)} 字符")

payload = {
    "model": "glm-4-voice",
    "messages": [{"role": "user", "content": long_input}]
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()

if "error" in result:
    print(f"❌ 错误:")
    print(json.dumps(result["error"], indent=2, ensure_ascii=False)[:500])
else:
    print("✅ 请求成功")
    print(f"状态码: {response.status_code}")

## 9. 综合测试总结

In [ ]:
# 列出所有生成的文件
print("=== 生成的文件汇总 ===" + "\n")

audio_files = [f for f in os.listdir('.') if f.endswith('.wav')]

if audio_files:
    print("🎵 音频文件:")
    for f in sorted(audio_files):
        size = os.path.getsize(f)
        print(f"   📁 {f} ({size} bytes)")
else:
    print("📂 没有生成音频文件")

print("\n✅ 测试完成！")

## 附录：如何获取音频文件的 base64 编码

### 方法 1：使用 Python

In [ ]:
# 将音频文件编码为 base64
def encode_audio_file_example(file_path):
    with open(file_path, "rb") as f:
        base64_data = base64.b64encode(f.read()).decode('utf-8')
    return base64_data

# 保存 base64 到文件
def save_base64_to_file_example(base64_data, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(base64_data)
    print(f"Base64 数据已保存到: {output_file}")

# 从文件读取 base64
def read_base64_from_file_example(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        return f.read()

# 解码 base64 并保存为音频文件
def decode_and_save_audio_example(base64_data, output_path):
    audio_data = base64.b64decode(base64_data)
    with open(output_path, "wb") as f:
        f.write(audio_data)
    print(f"音频已保存到: {output_path}")

print("✅ 示例函数定义完成")

### 方法 2：使用命令行

In [ ]:
# Linux / Mac
# base64 audio.wav > audio.txt

# Windows PowerShell
# [Convert]::ToBase64String([IO.File]::ReadAllBytes("audio.wav")) | Out-File -Encoding utf8 audio.txt

# 在线工具
# 也可以使用在线 base64 编码/解码工具

print("命令行示例已注释显示")